# Paso 1: Importar Librerías de Python


In [1]:
#pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163479 sha256=1a0c67a0e9bfe366991a9ed3c55d4543b7cb0404dfe99400fd2240f089643788
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
# Análisis de datos y manipulación
import pandas as pd
import numpy as np
import random


# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Utilidades de fechas
from datetime import datetime

# Integración con Google Colab
from google.colab import drive

# Machine Learning: Preprocesamiento y división de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Machine Learning: Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

# Machine Learning: Modelado y validación
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

# Deep Learning: Construcción de modelos y capas
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout, Dot,Lambda

# Deep Learning: Optimización y regularización
from keras.optimizers import Adam
from keras.regularizers import l2

# Deep Learning: Callbacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


# Paso 2: Lectura del Dataset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')

<ipython-input-4-9e77e6115feb>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')


In [5]:
df.shape

(4800571, 10)

In [6]:
sentimiento = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/sentimientos_productos.csv')

In [7]:
sentimiento.shape

(154026, 4)

# Modelo de Filtrado Colaborativo usando Embedding con Keras:

Funcionamiento:

Los embeddings son representaciones vectoriales densas y de baja dimensión de ítems y/o usuarios. Estas representaciones capturan relaciones semánticas entre ítems o entre usuarios.

Un método popular para generar embeddings es la factorización de matrices, como la descomposición en valores singulares (SVD). En el contexto de sistemas de recomendación, se busca factorizar la matriz de interacciones usuario-ítem en dos matrices más pequeñas (una para los usuarios y otra para los ítems) cuyo producto aproximado reproduce la matriz original lo mejor posible.

Una vez que se han obtenido los embeddings, la predicción de una calificación o interacción entre un usuario e ítem se realiza tomando el producto escalar entre sus embeddings respectivos.

Ventajas:

Simplicidad y eficiencia en términos computacionales.

Puede manejar grandes conjuntos de datos debido a su naturaleza de baja dimensión.

Es efectivo para capturar patrones subyacentes en los datos.

Desventajas:

No tiene en cuenta características adicionales de usuarios o ítems.

Dificultades para manejar nuevos ítems o usuarios (problema de arranque en frío).

El código ahora incluye regularización en los embeddings, un callback para detener el entrenamiento si no hay mejoras (early stopping) y otro para reducir la tasa de aprendizaje si el error de validación no mejora (reduce learning rate on plateau).

In [8]:
data = df

In [9]:
# Crear un LabelEncoder para cada columna
reviewerID_encoder = LabelEncoder()
asin_encoder = LabelEncoder()

# Ajustar y transformar las columnas
data['reviewerID_encoded'] = reviewerID_encoder.fit_transform(data['reviewerID'])
data['asin_encoded'] = asin_encoder.fit_transform(data['asin'])

# Número de usuarios únicos y número de ítems únicos
n_users = len(np.unique(data['reviewerID_encoded']))
n_items = len(np.unique(data['asin_encoded']))

# Dimensiones del embedding
embedding_dim = 10

# Entradas
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

# Embeddings con regularización
user_embedding = Embedding(n_users, embedding_dim, embeddings_regularizer=l2(1e-6))(user_input)
item_embedding = Embedding(n_items, embedding_dim, embeddings_regularizer=l2(1e-6))(item_input)

# Producto punto para predecir la valoración/rating
merged = Dot(axes=2)([user_embedding, item_embedding])
merged = Flatten()(merged)

# Agregar una capa de activación sigmoid y escalar a [0, 5]
output = Dense(1, activation='sigmoid')(merged)  # Función de activación sigmoid
output = Lambda(lambda x: x * 5)(output)  # Escalar las predicciones al rango [0, 5]

# Modelo
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

verbose=0: No muestra ninguna barra de progreso ni métricas.

verbose=1: Muestra una barra de progreso y actualiza las métricas después de cada lote.

verbose=2: Muestra las métricas después de cada época, pero no muestra la barra de progreso.

In [10]:
from sklearn.model_selection import train_test_split

# Crear los arrays con los datos
user_ids = data['reviewerID_encoded'].values
item_ids = data['asin_encoded'].values
ratings = data['overall'].values

# División en entrenamiento y prueba
(user_ids_temp, user_ids_test, item_ids_temp, item_ids_test, ratings_temp, ratings_test) = train_test_split(
    user_ids, item_ids, ratings, test_size=0.2, random_state=42)

# División del conjunto de entrenamiento en entrenamiento y validación
(user_ids_train, user_ids_val, item_ids_train, item_ids_val, ratings_train, ratings_val) = train_test_split(
    user_ids_temp, item_ids_temp, ratings_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [11]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [12]:
%%time
# Entrenar el modelo con el conjunto de entrenamiento
model.fit([user_ids_train, item_ids_train], ratings_train, validation_data=([user_ids_val, item_ids_val], ratings_val), epochs=500,
          batch_size=80000, verbose=1, callbacks=[early_stop, reduce_lr])

Epoch 1/500
37/37 [==============================] - 6s 18ms/step - loss: 4.3942 - mae: 1.9620 - val_loss: 4.3156 - val_mae: 1.9450 - lr: 0.0010
Epoch 2/500
37/37 [==============================] - 0s 11ms/step - loss: 4.2341 - mae: 1.9260 - val_loss: 4.1582 - val_mae: 1.9092 - lr: 0.0010
Epoch 3/500
37/37 [==============================] - 0s 11ms/step - loss: 4.0801 - mae: 1.8904 - val_loss: 4.0075 - val_mae: 1.8739 - lr: 0.0010
Epoch 4/500
37/37 [==============================] - 0s 11ms/step - loss: 3.9330 - mae: 1.8554 - val_loss: 3.8638 - val_mae: 1.8392 - lr: 0.0010
Epoch 5/500
37/37 [==============================] - 0s 11ms/step - loss: 3.7923 - mae: 1.8209 - val_loss: 3.7265 - val_mae: 1.8049 - lr: 0.0010
Epoch 6/500
37/37 [==============================] - 0s 11ms/step - loss: 3.6573 - mae: 1.7866 - val_loss: 3.5945 - val_mae: 1.7709 - lr: 0.0010
Epoch 7/500
37/37 [==============================] - 0s 11ms/step - loss: 3.5254 - mae: 1.7520 - val_loss: 3.4658 - val_mae: 1.736

In [13]:
y_true = ratings_test
predictions = model.predict([user_ids_test, item_ids_test])

30004/30004 [==============================] - 41s 1ms/step


In [14]:
# Evaluar el modelo con el conjunto de prueba
loss, mae = model.evaluate([user_ids_test, item_ids_test], ratings_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test MAE: {mae:.4f}')

30004/30004 [==============================] - 77s 3ms/step - loss: 1.4760 - mae: 0.7858
Test Loss: 1.4760
Test MAE: 0.7858


In [15]:
# MSE
mse = mean_squared_error(y_true, predictions)
print(f"MSE: {mse:.4f}")

# RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

# MAE
mae = mean_absolute_error(y_true, predictions)
print(f"MAE: {mae:.4f}")

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((y_true - predictions.flatten()) / y_true)) * 100
print(f"MAPE: {mape:.2f}%")

MSE: 1.3905
RMSE: 1.1792
MAE: 0.7858
MAPE: 34.94%


# Recomedanción

In [16]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id_test = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Identificar los ítems que el usuario ha calificado en el conjunto de prueba
indices_usuario = np.where(user_ids_test == usuario_id_test_encoded)[0]
items_calificados_test_encoded = item_ids_test[indices_usuario]
calificaciones_reales_test = ratings_test[indices_usuario]

# Predecir las calificaciones para estos ítems
calificaciones_predichas = model.predict([np.array([usuario_id_test_encoded] * len(items_calificados_test_encoded)), items_calificados_test_encoded])

# Decodificar los ítems para obtener los ID originales
items_calificados_test = asin_encoder.inverse_transform(items_calificados_test_encoded)

# Comparar las calificaciones predichas con las reales
comparacion = list(zip(items_calificados_test, calificaciones_reales_test, calificaciones_predichas.flatten()))

# Suponiendo que 'items_calificados_test' contiene los IDs originales de los ítems
recomendaciones_df = pd.DataFrame({
    'asin': items_calificados_test,
    'Calificación Real': calificaciones_reales_test,
    'Calificación Predicha': calificaciones_predichas.flatten()
})

recomendaciones_df = recomendaciones_df.merge(sentimiento[['asin', 'cluster']], on='asin', how='left')
recomendaciones_df.rename(columns={'cluster': 'Sentimiento'}, inplace=True)

# Mostrar las recomendaciones
print(f"Recomendaciones para el usuario {usuario_id_test}:")
for index, row in recomendaciones_df.iterrows():
    print(f"Ítem: {row['asin']}, Calificación Real: {row['Calificación Real']}, Calificación Predicha: {row['Calificación Predicha']}, Sentimiento: {row['Sentimiento']}")

1/1 [==============================] - 0s 29ms/step
Recomendaciones para el usuario A2N3UF4KPCZFXB:
Ítem: B0047RL1B4, Calificación Real: 3.0, Calificación Predicha: 4.322533130645752, Sentimiento: Positivo
Ítem: B000BIGOY2, Calificación Real: 5.0, Calificación Predicha: 4.604306697845459, Sentimiento: Neutro


In [17]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Lista de todos los ítems únicos en el dataset
items_unicos = np.unique(data['asin'].values)

# Ítems que el usuario ya ha calificado
items_calificados = data[data['reviewerID_encoded'] == usuario_id_test_encoded]['asin'].tolist()

# Ítems para predecir
items_a_predecir = [item for item in items_unicos if item not in items_calificados]

# Convierte los ítems a su representación codificada
items_a_predecir_encoded = asin_encoder.transform(items_a_predecir)

# Predecir la calificación para cada ítem no calificado
predicciones = model.predict([np.array([usuario_id_test_encoded] * len(items_a_predecir_encoded)), items_a_predecir_encoded])

# Emparejar cada ítem con su predicción
items_predicciones = list(zip(items_a_predecir, predicciones.flatten()))

# Ordenar por la calificación predicha más alta
items_recomendados = sorted(items_predicciones, key=lambda x: x[1], reverse=True)

# Tomar los top N recomendaciones
top_n = 5
top_recomendaciones = items_recomendados[:top_n]

# Crear un DataFrame a partir de las recomendaciones
recomendaciones_df = pd.DataFrame(top_recomendaciones, columns=['asin', 'Calificación Predicha'])

# Unir con el DataFrame 'sentimiento' para obtener el sentimiento asociado a cada ítem
recomendaciones_df = recomendaciones_df.merge(sentimiento[['asin', 'cluster']], on='asin', how='left')

# Renombrar la columna 'cluster' a 'Sentimiento' para claridad
recomendaciones_df.rename(columns={'cluster': 'Sentimiento'}, inplace=True)

# Mostrar las recomendaciones con el sentimiento
print(f"Recomendaciones para el usuario {usuario_id}:")
for index, row in recomendaciones_df.iterrows():
    print(f"Ítem: {row['asin']}, Calificación Predicha: {row['Calificación Predicha']}, Sentimiento: {row['Sentimiento']}")

4814/4814 [==============================] - 7s 1ms/step
Recomendaciones para el usuario A1J9MZ1T69YPKO:
Ítem: B002QG6P9G, Calificación Predicha: 4.981610298156738, Sentimiento: Positivo
Ítem: B0002ZW5UQ, Calificación Predicha: 4.977645397186279, Sentimiento: Neutro
Ítem: B0013U0EYI, Calificación Predicha: 4.976828575134277, Sentimiento: Neutro
Ítem: B002K6F79Q, Calificación Predicha: 4.973778247833252, Sentimiento: Neutro
Ítem: B000NG80GM, Calificación Predicha: 4.9720458984375, Sentimiento: Neutro


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa con Keras:

Funcionamiento:

Estos sistemas toman características de los ítems y/o usuarios y las pasan a través de una o varias capas de neuronas para obtener una predicción.
Las redes neuronales son capaces de capturar interacciones no lineales entre características, lo que las hace poderosas para tareas de modelado complejas.

Ventajas:

Capacidad de modelar relaciones no lineales.

Flexibilidad para incorporar múltiples fuentes de datos o características.

Puede manejar arranques en frío al incorporar características de nuevos ítems o usuarios.

Desventajas:

Mayor costo computacional en comparación con los sistemas basados únicamente en embeddings.

Riesgo de sobreajuste si no se tiene un conjunto de datos lo suficientemente grande.

Ahora el código incluye regularización en los embeddings, early stopping para detener el entrenamiento si el modelo deja de mejorar en el conjunto de validación, y reducción de la tasa de aprendizaje si el error en el conjunto de validación no mejora después de algunas épocas. Estas adiciones deberían ayudar a mejorar la capacidad de generalización del modelo.

In [18]:
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
hidden_units = [128, 64, 32]
dropout_rate = 0.2
l2_reg = 1e-6

# Arquitectura del modelo
user_input = Input(shape=[1], name='user_input')
item_input = Input(shape=[1], name='item_input')
concat = Concatenate()([user_input, item_input])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu', kernel_regularizer=l2(l2_reg))(dense)
    dense = Dropout(dropout_rate)(dense)

# Agregar una capa de activación sigmoid y escalar a [0, 5]
output = Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_reg))(dense)  # Función de activación sigmoid
output = Lambda(lambda x: x * 5)(output)  # Escalar las predicciones al rango [0, 5]

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

In [19]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [20]:
# Datos para entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# División en entrenamiento y prueba
(user_ids_temp, user_ids_test, item_ids_temp, item_ids_test, ratings_temp, ratings_test) = train_test_split(
    user_ids, item_ids, ratings, test_size=0.2, random_state=42)

# División del conjunto de entrenamiento en entrenamiento y validación
(user_ids_train, user_ids_val, item_ids_train, item_ids_val, ratings_train, ratings_val) = train_test_split(
    user_ids_temp, item_ids_temp, ratings_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [21]:
%%time
# Entrenamiento del modelo
history = model.fit([user_ids_train, item_ids_train], ratings_train,
    epochs=500,validation_data=([user_ids_val, item_ids_val], ratings_val),
    batch_size=80000,verbose=1,callbacks=[early_stop, reduce_lr])

Epoch 1/500
37/37 [==============================] - 3s 18ms/step - loss: 1.9356 - val_loss: 1.8981 - lr: 0.0010
Epoch 2/500
37/37 [==============================] - 0s 13ms/step - loss: 1.9094 - val_loss: 1.8981 - lr: 0.0010
Epoch 3/500
37/37 [==============================] - 0s 13ms/step - loss: 1.9092 - val_loss: 1.8981 - lr: 0.0010
Epoch 4/500
37/37 [==============================] - 0s 13ms/step - loss: 1.9091 - val_loss: 1.8981 - lr: 0.0010
Epoch 5/500
37/37 [==============================] - 0s 13ms/step - loss: 1.9090 - val_loss: 1.8981 - lr: 0.0010
Epoch 6/500
37/37 [==============================] - 0s 13ms/step - loss: 1.9091 - val_loss: 1.8981 - lr: 0.0010
Epoch 7/500
37/37 [==============================] - 0s 13ms/step - loss: 1.9091 - val_loss: 1.8981 - lr: 0.0010
Epoch 8/500
37/37 [==============================] - 0s 13ms/step - loss: 1.9091 - val_loss: 1.8981 - lr: 0.0010
Epoch 9/500
37/37 [==============================] - 0s 13ms/step - loss: 1.9090 - val_loss: 1.8

In [22]:
# Evaluar el modelo
y_true = ratings_test
predictions = model.predict([user_ids_test, item_ids_test])

30004/30004 [==============================] - 45s 1ms/step


In [23]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# Funciones MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 1.91015555428256
RMSE: 1.3820837725270345
MAE: 0.7401227977898481
MAPE: 40.77108911682409%


# Recomendación

In [24]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id_test = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Identificar los ítems que el usuario ha calificado en el conjunto de prueba
indices_usuario = np.where(user_ids_test == usuario_id_test_encoded)[0]
items_calificados_test_encoded = item_ids_test[indices_usuario]
calificaciones_reales_test = ratings_test[indices_usuario]

# Predecir las calificaciones para estos ítems
calificaciones_predichas = model.predict([np.array([usuario_id_test_encoded] * len(items_calificados_test_encoded)), items_calificados_test_encoded])

# Decodificar los ítems para obtener los ID originales
items_calificados_test = asin_encoder.inverse_transform(items_calificados_test_encoded)

# Comparar las calificaciones predichas con las reales
comparacion = list(zip(items_calificados_test, calificaciones_reales_test, calificaciones_predichas.flatten()))

# Suponiendo que 'items_calificados_test' contiene los IDs originales de los ítems
recomendaciones_df = pd.DataFrame({
    'asin': items_calificados_test,
    'Calificación Real': calificaciones_reales_test,
    'Calificación Predicha': calificaciones_predichas.flatten()
})

recomendaciones_df = recomendaciones_df.merge(sentimiento[['asin', 'cluster']], on='asin', how='left')
recomendaciones_df.rename(columns={'cluster': 'Sentimiento'}, inplace=True)

# Mostrar las recomendaciones
print(f"Recomendaciones para el usuario {usuario_id_test}:")
for index, row in recomendaciones_df.iterrows():
    print(f"Ítem: {row['asin']}, Calificación Real: {row['Calificación Real']}, Calificación Predicha: {row['Calificación Predicha']}, Sentimiento: {row['Sentimiento']}")

1/1 [==============================] - 0s 21ms/step
Recomendaciones para el usuario AQZ8ATLZ889GB:
Ítem: B00MLR65VS, Calificación Real: 3.0, Calificación Predicha: 5.0, Sentimiento: Neutro
Ítem: B00R1OAUNA, Calificación Real: 5.0, Calificación Predicha: 5.0, Sentimiento: Neutro
Ítem: B010TX3RBQ, Calificación Real: 5.0, Calificación Predicha: 5.0, Sentimiento: Positivo
Ítem: B00MOIPERK, Calificación Real: 5.0, Calificación Predicha: 5.0, Sentimiento: Positivo


In [25]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Lista de todos los ítems únicos en el dataset
items_unicos = np.unique(data['asin'].values)

# Ítems que el usuario ya ha calificado
items_calificados = data[data['reviewerID_encoded'] == usuario_id_test_encoded]['asin'].tolist()

# Ítems para predecir
items_a_predecir = [item for item in items_unicos if item not in items_calificados]

# Convierte los ítems a su representación codificada
items_a_predecir_encoded = asin_encoder.transform(items_a_predecir)

# Predecir la calificación para cada ítem no calificado
predicciones = model.predict([np.array([usuario_id_test_encoded] * len(items_a_predecir_encoded)), items_a_predecir_encoded])

# Emparejar cada ítem con su predicción
items_predicciones = list(zip(items_a_predecir, predicciones.flatten()))

# Ordenar por la calificación predicha más alta
items_recomendados = sorted(items_predicciones, key=lambda x: x[1], reverse=True)

# Tomar los top N recomendaciones
top_n = 5
top_recomendaciones = items_recomendados[:top_n]

# Crear un DataFrame a partir de las recomendaciones
recomendaciones_df = pd.DataFrame(top_recomendaciones, columns=['asin', 'Calificación Predicha'])

# Unir con el DataFrame 'sentimiento' para obtener el sentimiento asociado a cada ítem
recomendaciones_df = recomendaciones_df.merge(sentimiento[['asin', 'cluster']], on='asin', how='left')

# Renombrar la columna 'cluster' a 'Sentimiento' para claridad
recomendaciones_df.rename(columns={'cluster': 'Sentimiento'}, inplace=True)

# Mostrar las recomendaciones con el sentimiento
print(f"Recomendaciones para el usuario {usuario_id}:")
for index, row in recomendaciones_df.iterrows():
    print(f"Ítem: {row['asin']}, Calificación Predicha: {row['Calificación Predicha']}, Sentimiento: {row['Sentimiento']}")

4814/4814 [==============================] - 7s 2ms/step
Recomendaciones para el usuario A2S0S3Z9JHPPFK:
Ítem: 0871167042, Calificación Predicha: 5.0, Sentimiento: Positivo
Ítem: 1519588135, Calificación Predicha: 5.0, Sentimiento: Neutro
Ítem: 1579652956, Calificación Predicha: 5.0, Sentimiento: Positivo
Ítem: 1936023857, Calificación Predicha: 5.0, Sentimiento: Positivo
Ítem: 5120053017, Calificación Predicha: 5.0, Sentimiento: Neutro


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa y Embedding con Keras:

Otra enfoque popular para filtrado colaborativo es utilizar redes neuronales multicapa, que esencialmente aprenden características no lineales de los datos. Vamos a construir un modelo que fusiona los embeddings de usuarios y artículos (por ejemplo, películas) en una red neuronal densa.

Este enfoque, que combina embeddings y redes neuronales multicapa, puede capturar interacciones más complejas y no lineales entre usuarios y artículos. Sin embargo, es crucial prestar atención al sobreajuste y asegurarse de que el modelo no esté simplemente memorizando los datos. Por lo tanto, es recomendable emplear técnicas de regularización, ajustar hiperparámetros y validar el rendimiento con un conjunto de datos de validación.

Funcionamiento:

Combina lo mejor de ambos mundos. Primero, se utilizan embeddings para convertir ítems y usuarios en representaciones vectoriales densas. Luego, estas representaciones se pasan a través de una red neuronal para hacer la predicción.

El proceso generalmente comienza con capas de embedding que convierten identificadores de usuarios e ítems en vectores. Estos vectores luego se pasan a través de capas densas para obtener la predicción final.

Ventajas:

Capacidad de capturar patrones subyacentes en los datos mediante embeddings y modelar interacciones no lineales mediante la red neuronal.

Flexibilidad para incorporar características adicionales.

Potencialmente más preciso que cualquiera de los otros dos métodos por separado.

Desventajas:

Mayor complejidad en el modelado y entrenamiento.

Mayor costo computacional.

Requiere un ajuste más cuidadoso y riesgo de sobreajuste si no se gestiona adecuadamente.

Regularización L2: Agregar una regularización L2 a las capas de embedding y densas para prevenir el sobreajuste.

Early Stopping: Añadir una callback de early stopping para detener el entrenamiento cuando no haya mejora en el conjunto de validación durante un cierto número de épocas.

Ajuste de la tasa de aprendizaje: Utilizar la callback ReduceLROnPlateau para reducir la tasa de aprendizaje cuando no haya mejora en el conjunto de validación.

Métricas adicionales: Puedes monitorizar otras métricas como el error absoluto medio (MAE) durante el entrenamiento.

In [26]:
# Convertir reviewerID y asin a índices numéricos secuenciales
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
n_latent_factors = 50
hidden_units = [128, 64]
dropout_rate = 0.2
l2_reg = 1e-4

# Entradas
user_input = Input(shape=[1], name='user_input')
item_input = Input(shape=[1], name='item_input')

# Embeddings con regularización L2
user_embedding = Embedding(n_users, n_latent_factors, embeddings_regularizer=l2(l2_reg), name='user_embedding')(user_input)
item_embedding = Embedding(n_items, n_latent_factors, embeddings_regularizer=l2(l2_reg), name='item_embedding')(item_input)

user_vec = Flatten(name='flatten_users')(user_embedding)
item_vec = Flatten(name='flatten_items')(item_embedding)
concat = Concatenate()([user_vec, item_vec])

# Capas densas
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu', kernel_regularizer=l2(l2_reg))(dense)
    dense = Dropout(dropout_rate)(dense)

# Agregar una capa de activación sigmoid y escalar a [0, 5]
output = Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_reg))(dense)  # Función de activación sigmoid
output = Lambda(lambda x: x * 5)(output)  # Escalar las predicciones al rango [0, 5]

# Modelo
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error', metrics=['mae'])

In [27]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [28]:
# Entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# División en entrenamiento y prueba
(user_ids_temp, user_ids_test, item_ids_temp, item_ids_test, ratings_temp, ratings_test) = train_test_split(
    user_ids, item_ids, ratings, test_size=0.2, random_state=42)

# División del conjunto de entrenamiento en entrenamiento y validación
(user_ids_train, user_ids_val, item_ids_train, item_ids_val, ratings_train, ratings_val) = train_test_split(
    user_ids_temp, item_ids_temp, ratings_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [29]:
%%time
history = model.fit([user_ids_train, item_ids_train], ratings_train,
                    epochs=500,validation_data=([user_ids_val, item_ids_val], ratings_val),
                    batch_size=80000,verbose=1,callbacks=[early_stop, reduce_lr])

Epoch 1/500
37/37 [==============================] - 4s 67ms/step - loss: 5.9615 - mae: 1.5502 - val_loss: 2.0601 - val_mae: 0.9238 - lr: 0.0010
Epoch 2/500
37/37 [==============================] - 2s 52ms/step - loss: 1.5950 - mae: 0.8480 - val_loss: 1.3922 - val_mae: 0.8746 - lr: 0.0010
Epoch 3/500
37/37 [==============================] - 2s 43ms/step - loss: 1.2299 - mae: 0.7847 - val_loss: 1.4081 - val_mae: 0.8116 - lr: 0.0010
Epoch 4/500
37/37 [==============================] - 2s 43ms/step - loss: 1.0883 - mae: 0.6935 - val_loss: 1.4304 - val_mae: 0.7991 - lr: 0.0010
Epoch 5/500
37/37 [==============================] - 2s 43ms/step - loss: 1.0284 - mae: 0.6577 - val_loss: 1.4506 - val_mae: 0.7929 - lr: 0.0010
Epoch 6/500
37/37 [==============================] - 2s 44ms/step - loss: 1.0039 - mae: 0.6384 - val_loss: 1.4697 - val_mae: 0.7905 - lr: 0.0010
Epoch 7/500
37/37 [==============================] - 2s 44ms/step - loss: 0.9856 - mae: 0.6246 - val_loss: 1.4819 - val_mae: 0.788

In [30]:
# Predicciones
y_true = ratings_test
predictions = model.predict([user_ids_test, item_ids_test])

30004/30004 [==============================] - 45s 2ms/step


In [31]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 1.2274721416811742
RMSE: 1.1079134179533952
MAE: 0.8753315500893905
MAPE: 38.799056155574256%


# Recomendación

In [32]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id_test = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Identificar los ítems que el usuario ha calificado en el conjunto de prueba
indices_usuario = np.where(user_ids_test == usuario_id_test_encoded)[0]
items_calificados_test_encoded = item_ids_test[indices_usuario]
calificaciones_reales_test = ratings_test[indices_usuario]

# Predecir las calificaciones para estos ítems
calificaciones_predichas = model.predict([np.array([usuario_id_test_encoded] * len(items_calificados_test_encoded)), items_calificados_test_encoded])

# Decodificar los ítems para obtener los ID originales
items_calificados_test = asin_encoder.inverse_transform(items_calificados_test_encoded)

# Comparar las calificaciones predichas con las reales
comparacion = list(zip(items_calificados_test, calificaciones_reales_test, calificaciones_predichas.flatten()))

# Suponiendo que 'items_calificados_test' contiene los IDs originales de los ítems
recomendaciones_df = pd.DataFrame({
    'asin': items_calificados_test,
    'Calificación Real': calificaciones_reales_test,
    'Calificación Predicha': calificaciones_predichas.flatten()
})

recomendaciones_df = recomendaciones_df.merge(sentimiento[['asin', 'cluster']], on='asin', how='left')
recomendaciones_df.rename(columns={'cluster': 'Sentimiento'}, inplace=True)

# Mostrar las recomendaciones
print(f"Recomendaciones para el usuario {usuario_id_test}:")
for index, row in recomendaciones_df.iterrows():
    print(f"Ítem: {row['asin']}, Calificación Real: {row['Calificación Real']}, Calificación Predicha: {row['Calificación Predicha']}, Sentimiento: {row['Sentimiento']}")

1/1 [==============================] - 0s 19ms/step
Recomendaciones para el usuario A3MSML4BHRP3RC:
Ítem: B0058YYLEY, Calificación Real: 5.0, Calificación Predicha: 4.682139873504639, Sentimiento: Positivo


In [33]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Lista de todos los ítems únicos en el dataset
items_unicos = np.unique(data['asin'].values)

# Ítems que el usuario ya ha calificado
items_calificados = data[data['reviewerID_encoded'] == usuario_id_test_encoded]['asin'].tolist()

# Ítems para predecir
items_a_predecir = [item for item in items_unicos if item not in items_calificados]

# Convierte los ítems a su representación codificada
items_a_predecir_encoded = asin_encoder.transform(items_a_predecir)

# Predecir la calificación para cada ítem no calificado
predicciones = model.predict([np.array([usuario_id_test_encoded] * len(items_a_predecir_encoded)), items_a_predecir_encoded])

# Emparejar cada ítem con su predicción
items_predicciones = list(zip(items_a_predecir, predicciones.flatten()))

# Ordenar por la calificación predicha más alta
items_recomendados = sorted(items_predicciones, key=lambda x: x[1], reverse=True)

# Tomar los top N recomendaciones
top_n = 5
top_recomendaciones = items_recomendados[:top_n]

# Crear un DataFrame a partir de las recomendaciones
recomendaciones_df = pd.DataFrame(top_recomendaciones, columns=['asin', 'Calificación Predicha'])

# Unir con el DataFrame 'sentimiento' para obtener el sentimiento asociado a cada ítem
recomendaciones_df = recomendaciones_df.merge(sentimiento[['asin', 'cluster']], on='asin', how='left')

# Renombrar la columna 'cluster' a 'Sentimiento' para claridad
recomendaciones_df.rename(columns={'cluster': 'Sentimiento'}, inplace=True)

# Mostrar las recomendaciones con el sentimiento
print(f"Recomendaciones para el usuario {usuario_id}:")
for index, row in recomendaciones_df.iterrows():
    print(f"Ítem: {row['asin']}, Calificación Predicha: {row['Calificación Predicha']}, Sentimiento: {row['Sentimiento']}")

4814/4814 [==============================] - 7s 2ms/step
Recomendaciones para el usuario A88BPHHETU0SD:
Ítem: B00V5RV0ME, Calificación Predicha: 4.8303680419921875, Sentimiento: Positivo
Ítem: B00RLSCLJM, Calificación Predicha: 4.817205429077148, Sentimiento: Positivo
Ítem: B0007M11XQ, Calificación Predicha: 4.8141889572143555, Sentimiento: Positivo
Ítem: B00KKXCJQU, Calificación Predicha: 4.8032965660095215, Sentimiento: Positivo
Ítem: B000JFBHGW, Calificación Predicha: 4.796957492828369, Sentimiento: Positivo


Estas métricas te ayudarán a tener una mejor idea del desempeño de tu modelo. Por ejemplo:

MSE y RMSE son útiles cuando quieres penalizar grandes errores.

MAE te da una idea del error medio sin considerar la dirección del error.

MAPE es útil cuando quieres representar el error en términos porcentuales.

Para una evaluación completa, es recomendable utilizar un conjunto de validación aparte (es decir, no solo depender del validation_split). Esto asegura que estás evaluando el desempeño en datos que el modelo nunca ha visto durante el entrenamiento.

En resumen, la elección del tipo de sistema de recomendación dependerá de la naturaleza del conjunto de datos, las características disponibles, las capacidades computacionales y el tipo de relaciones o interacciones que se desean capturar. A menudo, un enfoque híbrido (como el sistema que combina embeddings y redes neuronales) ofrece un buen equilibrio entre precisión y eficiencia.